In [1]:
import requests, json
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import sys
import time
import requests, json
import numpy as np
import calendar
from datetime import datetime
import random
import glob

In [2]:
#converts numpy types to python types, otherwise json conversion produces an error. call json.dumps(***, cls=MyEncoder)
class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(MyEncoder, self).default(obj)


# returns strings in the conventional format for iot.ids
def idstr(idinput):
    return(str(idinput).lower().replace(" ", "_").replace("/", "_").replace("ä", "ae").replace("ö", "oe").replace("ü", "ue").replace("ß", "ss"))


# returns the first 7 digits of the sha1 hash of the input string
def hashfunc(inputstring, printme):
    returnhash= hashlib.sha1(bytes(str(inputstring), 'utf-8')).hexdigest()[0:7]
    if printme == True:
        print("Converting '" + str(inputstring) + "' to hash '" + str(returnhash) +"'")
    return(returnhash)

url="http://smartaqnet-dev.teco.edu/v1.0"

In [36]:
#requests.delete("http://smartaqnet-dev.teco.edu:8080/FROST-Server/v1.0/Things")
#requests.delete("http://smartaqnet-dev.teco.edu:8080/FROST-Server/v1.0/Sensors")
#requests.delete("http://smartaqnet-dev.teco.edu:8080/FROST-Server/v1.0/Locations")
#requests.delete("http://smartaqnet-dev.teco.edu:8080/FROST-Server/v1.0/ObservedProperties")
#requests.delete("http://smartaqnet-dev.teco.edu:8080/FROST-Server/v1.0/FeaturesOfInterest")
#requests.delete("http://smartaqnet-dev.teco.edu:8080/FROST-Server/v1.0/Datastreams")
#requests.delete("http://smartaqnet-dev.teco.edu:8080/FROST-Server/v1.0/Observations")

<Response [200]>

## Patching Template

In [3]:
# Things, Datasteams, Locations, ...
category_to_patch = "Things"

#saqn:...
entity_to_patch_id = "saqn:t:d42cbb8"


In [5]:
#optional get request for template on what to patch
query = ""

loaded_data = json.loads(requests.get("http://smartaqnet-dev.teco.edu/v1.0/" + category_to_patch + "('" + entity_to_patch_id + "')" + query).text)
loaded_data

{'name': 'Station Augsburg/Königsplatz',
 'description': 'A station measuring -Particulate air pollutants- -Gaseous air pollutants- -Pollutants in suspended particulate matter- -Pollutants in PM10- -Meteorological measurands-',
 'properties': {'dem_status': 'M',
  'station_code': 'DEBY006',
  'station_local_code': 'DEBY006',
  'station_name': 'Augsburg/Königsplatz',
  'station_start_date': 19750101,
  'station_end_date': 'nan',
  'station_latitude_d': 48.364589,
  'station_latitude_dms': '+048.21.52',
  'station_longitude_d': 10.895029,
  'station_longitude_dms': '+010.53.42',
  'station_altitude': 492.0,
  'type_of_station': 'traffic',
  'station_type_of_area': 'urban',
  'station_ozone_classification': 'urban',
  'station_subcat_rural_background': 'unknown',
  'network_code': 'DE007A',
  'operator_url': 'lfu.bayern.de'},
 'Datastreams@iot.navigationLink': "https://smartaqnet-dev.teco.edu/v1.0/Things('saqn%3At%3Ad42cbb8')/Datastreams",
 'MultiDatastreams@iot.navigationLink': "https://

In [12]:
# loaded_data["properties"]["station_code"]
loaded_data["properties"]["license"] = {"type": "", "owner": "Umweltbundesamt"}
loaded_data["properties"]

{'dem_status': 'M',
 'station_code': 'DEBY006',
 'station_local_code': 'DEBY006',
 'station_name': 'Augsburg/Königsplatz',
 'station_start_date': 19750101,
 'station_end_date': 'nan',
 'station_latitude_d': 48.364589,
 'station_latitude_dms': '+048.21.52',
 'station_longitude_d': 10.895029,
 'station_longitude_dms': '+010.53.42',
 'station_altitude': 492.0,
 'type_of_station': 'traffic',
 'station_type_of_area': 'urban',
 'station_ozone_classification': 'urban',
 'station_subcat_rural_background': 'unknown',
 'network_code': 'DE007A',
 'operator_url': 'lfu.bayern.de',
 'license': {'type': '', 'owner': 'Umweltbundesamt'}}

In [13]:
#what to patch?

patch_what ={"properties": loaded_data["properties"]}


#send patch request
resp_dev=requests.patch("http://smartaqnet-dev.teco.edu/v1.0/" + category_to_patch + "('" + entity_to_patch_id + "')",json=patch_what)
# resp=requests.patch("http://smartaqnet.teco.edu/v1.0/" + category_to_patch + "('" + entity_to_patch_id + "')",json=patch_what)

print(resp_dev)
# print(resp)

<Response [403]>


## Patching Locations for Workshop Nodes

In [11]:
#nr=11
#a_location=json.loads(requests.get("http://smartaqnet.teco.edu/v1.0/Locations?$select=@iot.id,location").text)["value"][nr]
#a_location

a_location=json.loads(requests.get("https://smartaqnet.teco.edu/v1.0/Locations('geo%3A10.94%2C48.38%2Cnan')?$select=@iot.id,location").text)
a_location


#breit ist das große, lang das kleine
#im json muss das kleine zuerst: [lang, breit]

{'location': {'type': 'Point', 'coordinates': [48.37616, 10.94464]},
 '@iot.id': 'geo:10.94,48.38,nan'}

In [8]:
breit=a_location["location"]["coordinates"][0]
lang=a_location["location"]["coordinates"][1]

In [9]:
requests.patch("http://smartaqnet.teco.edu/v1.0/Locations('" + a_location["@iot.id"] + "')",json={"location": {"type": "Point", "coordinates": [lang,breit]}})

<Response [403]>

In [248]:
json.loads(requests.get("http://smartaqnet.teco.edu/v1.0/Locations('" + str(a_location["@iot.id"]) + "')").text)

{'name': 'Tk',
 'description': '',
 'encodingType': 'application/vnd.geo+json',
 'location': {'type': 'Point', 'coordinates': [-3.0, 45.0]},
 'HistoricalLocations@iot.navigationLink': "http://smartaqnet.teco.edu/v1.0/Locations('geo%3A-3.00%2C45.00%2Cnan')/HistoricalLocations",
 'Things@iot.navigationLink': "http://smartaqnet.teco.edu/v1.0/Locations('geo%3A-3.00%2C45.00%2Cnan')/Things",
 '@iot.id': 'geo:-3.00,45.00,nan',
 '@iot.selfLink': "http://smartaqnet.teco.edu/v1.0/Locations('geo%3A-3.00%2C45.00%2Cnan')"}